In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
from collections import Counter
from pyinform import transfer_entropy
from pyinform import mutual_info
from statsmodels.tsa.stattools import grangercausalitytests
colors_to_numbers = {'gru': 0, 'pty': 1, 'jfk': 2, 'lax': 3, 'las': 4, 'lis': 5, 'bru': 6, 'hkg': 7, 'icn': 8, 'mex': 9}

In [120]:
#transfer entropy State Transition

entropy=0
mutual_info_state_transitions_same_timestep = [0] * 400
num_df = 15

dataframes = []
time_of_transitions = []
state_of_transitions = []

for baseline_cell in range(388,389):#401
    mutual_info_state_transitions_same_timestep = [0] * 400
    print('BaseLine Cell: ',baseline_cell)
    for df_counter in range(0, num_df):
        if df_counter % 20 == 0:
            print(df_counter)
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers)
        baseline_cell_colors = []
        baseline_cell_transition_index =[]
        X = list(df.iloc[:,baseline_cell])
        baseline_cell_colors.append(X[0])
        baseline_cell_transition_index.append(0)
        for j in range(1,len(X)):
            if X[j] != X[j-1]:
                baseline_cell_colors.append(X[j]) #appends the color to the list at the time of the change 
                baseline_cell_transition_index.append(j)
        dataframes.append(df_counter)
        time_of_transitions.append(baseline_cell_transition_index)
        state_of_transitions.append(baseline_cell_colors)
        

BaseLine Cell:  388
0


In [126]:
time_of_transitions

[[0, 50, 623],
 [0, 77, 578],
 [0, 149, 478, 501, 999, 1620],
 [0, 1, 659],
 [0, 19, 499, 1626],
 [0, 47, 568],
 [0, 18, 252, 669],
 [0, 70, 449],
 [0, 97, 422, 431, 971, 1585],
 [0, 8, 208, 419],
 [0, 287, 320, 600, 931],
 [0, 32, 94, 321, 420],
 [0, 36, 919],
 [0, 102, 455, 1580],
 [0, 52, 102, 548, 679]]

In [134]:
def reorg_dataframe_at_a_timestep(dataframe):
    q1,q2,q3,q4 = X[:100],X[100:200],X[200:300],X[300:400]
    matrix_Q1, matrix_Q2, matrix_Q3, matrix_Q4 = np.array(q1).reshape(10, 10),np.array(q2).reshape(10, 10),np.array(q3).reshape(10, 10), np.array(q4).reshape(10, 10)
    top_half = np.concatenate((matrix_Q2, matrix_Q1), axis=1)  # Q1 and Q2 side-by-side
    bottom_half = np.concatenate((matrix_Q3, matrix_Q4), axis=1)  # Q3 and Q4 side-by-side
    z = np.concatenate((top_half, bottom_half), axis=0)
    return z

In [135]:
def get_neighborhood_sum(dataframe,row,col):
    z = dataframe
    neighbor_TL = z[row-1][col-1]
    neighbor_T = z[row-1][col]
    neighbor_TR = z[row-1][(col+1)%20]
    neighbor_L = z[row][col-1]
    neighbor_R = z[row][(col+1)%20]
    neighbor_BL = z[(row+1)%20][col-1]
    neighbor_B = z[(row+1)%20][col]
    neighbor_BR = z[(row+1)%20][(col+1)%20]
    return (neighbor_TL + neighbor_T +neighbor_TR + neighbor_L + neighbor_R + neighbor_BL + neighbor_B + neighbor_BR)


In [142]:
for dataframe in dataframes:
    df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(dataframe)+'.csv').replace(colors_to_numbers)
    print(dataframe)
    print(time_of_transitions[dataframe])
    print(state_of_transitions[dataframe])
    
    for timestep in time_of_transitions[dataframe]:
        print('timestep of transition is',timestep)
        sum = 0
        if timestep == 0: 
            X = list(df.iloc[timestep,:])[1:]
        else:
            X = list(df.iloc[timestep-1,:])[1:]
        Z = reorg_dataframe_at_a_timestep(X) 
        sum+=get_neighborhood_sum(Z,18,9)
        print('sum is',sum)    
    print('----------------------------')

0
[0, 50, 623]
[7, 6, 7]
timestep of transition is 0
sum is 42
timestep of transition is 50
sum is 43
timestep of transition is 623
sum is 52
----------------------------
1
[0, 77, 578]
[5, 7, 8]
timestep of transition is 0
sum is 34
timestep of transition is 77
sum is 37
timestep of transition is 578
sum is 45
----------------------------
2
[0, 149, 478, 501, 999, 1620]
[4, 5, 6, 5, 6, 7]
timestep of transition is 0
sum is 26
timestep of transition is 149
sum is 45
timestep of transition is 478
sum is 51
timestep of transition is 501
sum is 51
timestep of transition is 999
sum is 50
timestep of transition is 1620
sum is 54
----------------------------
3
[0, 1, 659]
[2, 6, 7]
timestep of transition is 0
sum is 46
timestep of transition is 1
sum is 46
timestep of transition is 659
sum is 47
----------------------------
4
[0, 19, 499, 1626]
[4, 5, 6, 7]
timestep of transition is 0
sum is 38
timestep of transition is 19
sum is 36
timestep of transition is 499
sum is 38
timestep of transit

In [287]:
####this is the code that breaks up the dataframe into the format that we need in order to calculate neighbors for any point in time 



df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_0.csv').replace(colors_to_numbers)
###############time,cell#############
X = (df.iloc[0,:].tolist())[1:]
X
q1,q2,q3,q4 = X[:100],X[100:200],X[200:300],X[300:400]
matrix_Q1, matrix_Q2, matrix_Q3, matrix_Q4 = np.array(q1).reshape(10, 10),np.array(q2).reshape(10, 10),np.array(q3).reshape(10, 10), np.array(q4).reshape(10, 10)

top_half = np.concatenate((matrix_Q2, matrix_Q1), axis=1)  # Q1 and Q2 side-by-side
bottom_half = np.concatenate((matrix_Q3, matrix_Q4), axis=1)  # Q3 and Q4 side-by-side
z = np.concatenate((top_half, bottom_half), axis=0)

#cell to analyze at position 17,8

sum = 0

row = 19
col = 0 
neighbor_TL = z[row-1][col-1]
neighbor_T = z[row-1][col]
neighbor_TR = z[row-1][(col+1)%20]
neighbor_L = z[row][col-1]
neighbor_R = z[row][(col+1)%20]
neighbor_BL = z[(row+1)%20][col-1]
neighbor_B = z[(row+1)%20][col]
neighbor_BR = z[(row+1)%20][(col+1)%20]


#print(neighbor_TL,neighbor_T,neighbor_TR,'\n',neighbor_L, 'CELL', neighbor_R, '\n', neighbor_BL,neighbor_B,neighbor_BR)
data_for_granger = df[['Q3-pos201','Q3-pos284']]
data_for_granger



granger_test = grangercausalitytests(data_for_granger,2)
granger_test




Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.6451  , p=0.1040  , df_denom=1997, df_num=1
ssr based chi2 test:   chi2=2.6490  , p=0.1036  , df=1
likelihood ratio test: chi2=2.6473  , p=0.1037  , df=1
parameter F test:         F=2.6451  , p=0.1040  , df_denom=1997, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.3605  , p=0.2568  , df_denom=1994, df_num=2
ssr based chi2 test:   chi2=2.7279  , p=0.2556  , df=2
likelihood ratio test: chi2=2.7260  , p=0.2559  , df=2
parameter F test:         F=1.3605  , p=0.2568  , df_denom=1994, df_num=2


{1: ({'ssr_ftest': (2.645060490482713, 0.10402997626846884, 1997.0, 1),
   'ssr_chi2test': (2.649034041545031, 0.10361283952657788, 1),
   'lrtest': (2.647281243775069, 0.1037271615401371, 1),
   'params_ftest': (2.6450604904821358, 0.10402997626854114, 1997.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.3605423281457805, 0.2567596491091238, 1994.0, 2),
   'ssr_chi2test': (2.7279078374758425, 0.25564796470641155, 2),
   'lrtest': (2.7260482281308214, 0.25588577792129935, 2),
   'params_ftest': (1.3605423281316722, 0.2567596491127254, 1994.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])])}

In [ ]:
def reorg_dataframe_at_a_timestep(dataframe):
    

In [ ]:
entropy=0
mutual_info_state_transitions_same_timestep = [0] * 400
num_df = 199

for baseline_cell in range(1,401):#401
    mutual_info_state_transitions_same_timestep = [0] * 400
    print('BaseLine Cell: ',baseline_cell)
    for df_counter in range(0, num_df):
        if df_counter % 20 == 0:
            print(df_counter)
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers)

In [ ]:
#defining the ranges that we notice in the in the black box
#lets only test ofr the significance in one cell in s5

# let comp cell = row 17, cell 18 


df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_0.csv').replace(colors_to_numbers)

In [ ]:
#Optimized with GPT
#all changes 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
import os

# Function to load CSV data once to reduce I/O time
def load_csv_data(num_df, path_prefix, colors_to_numbers):
    dataframes = []
    for df_counter in range(num_df):
        csv_path = os.path.join(path_prefix, f'blackbox_{df_counter}.csv')
        df = pd.read_csv(csv_path).replace(colors_to_numbers)
        dataframes.append(df)
    return dataframes


# Function to compute transfer entropy for each baseline cell
def compute_transfer_entropy(baseline_cell, dataframes, num_df, colors_to_numbers):
    mutual_info_state_transitions_same_timestep = np.zeros(400)
    
    for df_counter, df in enumerate(dataframes):
        if df_counter % 20 == 0:
            print(f'Processing dataframe {df_counter} for baseline cell {baseline_cell}')
        
        baseline_cell_colors = list(df.iloc[:, baseline_cell])
        
        for i in range(1, df.shape[1]):
            comparator_cell_colors = list(df.iloc[:, i])
            mutual_info_state_transitions_same_timestep[i-1] += float(transfer_entropy(baseline_cell_colors, comparator_cell_colors, k=2))
    
    # Normalize by the number of dataframes
    mutual_info_state_transitions_same_timestep /= num_df
    mutual_info_state_transitions_same_timestep *= 100
    return mutual_info_state_transitions_same_timestep

# Plot results with seaborn
def plot_results(baseline_cell, results):
    data_100, data_200, data_300, data_400 = results[:100], results[100:200], results[200:300], results[300:400]
    q1, q2, q3, q4 = np.reshape(data_100, (10,10)), np.reshape(data_200, (10,10)), np.reshape(data_300, (10,10)), np.reshape(data_400, (10,10))
    
    figure, axis = plt.subplots(2, 2, figsize=(15, 15))
    plt.figure(figsize=(15, 15))
    sns.heatmap(q1, annot=True, fmt=".2f", cmap="YlGn", ax=axis[0,1] )
    axis[0,1].set_title("Q2")
    sns.heatmap(q2, annot=True, fmt=".2f", cmap="YlGn", ax=axis[0,0])
    axis[0,0].set_title("Q1")
    sns.heatmap(q3, annot=True, fmt=".2f", cmap="YlGn", ax=axis[1,0])
    axis[1,0].set_title("Q3")
    sns.heatmap(q4, annot=True, fmt=".2f", cmap="YlGn",  ax=axis[1,1])
    axis[1,1].set_title("Q4")
    
    figure.suptitle(f"State Transition Transfer Entropy#{baseline_cell-1}\nEntropies Multiplied by 100 for Graphical Representation")
    output_path = f'C:\\Users\\matth\\PhD\\Fall 2024\\SSIE 501-SSE 440\\Programming\\BlackBox1_Full_Transfer_Entropy\\Transfer_Entropy_{baseline_cell-1}.jpg'
    figure.savefig(output_path)
    plt.close(figure)

# Load all CSV files first to minimize I/O
csv_path_prefix = r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4'
colors_to_numbers = {'gru': 0, 'pty': 1, 'jfk': 2, 'lax': 3, 'las': 4, 'lis': 5, 'bru': 6, 'hkg': 7, 'icn': 8, 'mex': 9}  # Your colors to numbers mapping here
dataframes = load_csv_data(num_df=199, path_prefix=csv_path_prefix, colors_to_numbers=colors_to_numbers)

# Parallelize computation for each baseline cell using joblib
results = Parallel(n_jobs=-1)(delayed(compute_transfer_entropy)(baseline_cell, dataframes, 199, colors_to_numbers) for baseline_cell in range(1, 401))

# Plot results for each baseline cell
for baseline_cell, result in enumerate(results, start=1):
    print(f'Plotting results for baseline cell {baseline_cell}')
    plot_results(baseline_cell, result) 

In [451]:
def reorg_dataframe_at_a_timestep(data_frame_row_as_list):
    X = data_frame_row_as_list
    q1,q2,q3,q4 = X[:100],X[100:200],X[200:300],X[300:400]
    matrix_Q1, matrix_Q2, matrix_Q3, matrix_Q4 = np.array(q1).reshape(10, 10),np.array(q2).reshape(10, 10),np.array(q3).reshape(10, 10), np.array(q4).reshape(10, 10)
    top_half = np.concatenate((matrix_Q2, matrix_Q1), axis=1)  # Q1 and Q2 side-by-side
    bottom_half = np.concatenate((matrix_Q3, matrix_Q4), axis=1)  # Q3 and Q4 side-by-side
    z = np.concatenate((top_half, bottom_half), axis=0)
    return z

def get_neighborhood_sum(dataframe,row,col):
    z = dataframe
    neighbor_TL = z[row-1][col-1]
    neighbor_T = z[row-1][col]
    neighbor_TR = z[row-1][(col+1)%20]
    neighbor_L = z[row][col-1]
    neighbor_R = z[row][(col+1)%20]
    neighbor_BL = z[(row+1)%20][col-1]
    neighbor_B = z[(row+1)%20][col]
    neighbor_BR = z[(row+1)%20][(col+1)%20]
    return (neighbor_TL + neighbor_T +neighbor_TR + neighbor_L + neighbor_R + neighbor_BL + neighbor_B + neighbor_BR)

def baseline_cell_to_matrix(baseline_cell):
    baseline_cell = baseline_cell-1
    if baseline_cell<=100:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return baseline_cell//10%10, (baseline_cell%10)+10
    elif baseline_cell<=200:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return baseline_cell//10%10, (baseline_cell%10)
    elif baseline_cell<=300:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return (baseline_cell//10%10)+10, (baseline_cell%10)
    else:
        row = baseline_cell//10%10
        col = baseline_cell%10
        col_offset = 10
        return (baseline_cell//10%10)+10, (baseline_cell%10)+10
    return row,col

In [294]:
#retrying granger causuality in a non-moronic way 
df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_0.csv').replace(colors_to_numbers)

shifted = df['Q4-pos398'] != df['Q4-pos398'].shift(1)
shifted


# Specify the column where you want to detect shifts (e.g., column 'A')
state_after_transition = df[df['Q4-pos388'] != df['Q4-pos388'].shift()].index #grtabs all the locations where the row is not equal to the row before it 
print('The locations where the column value changes are',state_after_transition) #tells me the rows where the value updates
state_before_transition = state_after_transition.where(state_after_transition == 0, state_after_transition - 1)
print('The locations just before the column value changes',state_before_transition) #gets the values just before the row updates
txn_vals = df['Q4-pos388'].loc[state_before_transition].tolist()
txn_vals_after = df['Q4-pos388'].loc[state_after_transition].tolist()
# print('the values before time of transition are:    ',txn_vals)
print('the values after the time of transition are: ',txn_vals_after)

#print('the dataframe representing the entire state of the balck box at the moment before transition is:')
df = df.loc[state_before_transition]
df = df.drop(df.columns[0], axis=1)

sum = []
for i in range(0,df.shape[0]):
    Z = reorg_dataframe_at_a_timestep(df.iloc[i,:].tolist())
    sum.append((get_neighborhood_sum(Z,18,18)))

#now, i need to take this dataframe, convert it to a usable format, and grab the sum of the neighbors

print(txn_vals_after[1:]) # we need to take from 1 on becausse we are do not predict the first transisition, we only predict from the first transisiton onwards
print(sum[1:])

granger_test_data = pd.DataFrame({'list1': txn_vals_after, 'list2': sum})

granger_test = grangercausalitytests(granger_test_data,1)

The locations where the column value changes are Index([0, 210, 417, 970, 1082, 1137, 1565], dtype='int64')
The locations just before the column value changes Index([0, 209, 416, 969, 1081, 1136, 1564], dtype='int64')
the values after the time of transition are:  [5, 6, 7, 8, 7, 8, 7]
[6, 7, 8, 7, 8, 7]
[43, 48, 56, 55, 56, 54]

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.3797  , p=0.2206  , df_denom=3, df_num=1
ssr based chi2 test:   chi2=4.7594  , p=0.0291  , df=1
likelihood ratio test: chi2=3.5041  , p=0.0612  , df=1
parameter F test:         F=2.3797  , p=0.2206  , df_denom=3, df_num=1


In [310]:
df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_0.csv').replace(colors_to_numbers)

shifted1 = df.iloc[:,388] != df.iloc[:,388].shift(1)

shifted2 = df['Q4-pos389'] != df['Q4-pos389'].shift(1)
print(shifted1)
print(shifted2)


0        True
1       False
2       False
3       False
4       False
        ...  
1996    False
1997    False
1998    False
1999    False
2000    False
Name: Q4-pos387, Length: 2001, dtype: bool
0        True
1       False
2       False
3       False
4       False
        ...  
1996    False
1997    False
1998    False
1999    False
2000    False
Name: Q4-pos389, Length: 2001, dtype: bool


In [ ]:
num_df = 199
comparator_state = [0]*400
running_state_transitions = []
running_sums = [0]

for baseline_cell in range(355,356): #cells in range 1 --> 401
    print('BaseLine Cell: ',baseline_cell)
    sum = []
    row,col = baseline_cell_to_matrix(baseline_cell)
    print(row,col)
    for df_counter in range(0, num_df):
        df = pd.read_csv(r'C:\Users\matth\PhD\Fall 2024\SSIE 501-SSE 440\Programming\ScrapingOutput\DataCollector4\blackbox_'+str(df_counter)+'.csv').replace(colors_to_numbers) #first read the pdf
        #shifted = df['Q4-pos398'] != df['Q4-pos398'].shift(1)
        shifted =                 df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift(1)
        state_after_transition =  df[df.iloc[:,baseline_cell] != df.iloc[:,baseline_cell].shift()].index 
        state_before_transition = state_after_transition.where(state_after_transition == 0, state_after_transition - 1)
        txn_vals =                df.iloc[:,baseline_cell].loc[state_before_transition].tolist()
        txn_vals_after =          df.iloc[:,baseline_cell].loc[state_after_transition].tolist()
        
        df_cond = df.loc[state_before_transition]
        df_cond = df_cond.drop(df.columns[0], axis=1)

        

        for i in range(1,df_cond.shape[0]):
            #print(i)
            Z = reorg_dataframe_at_a_timestep(df_cond.iloc[i,:].tolist())
            
            sum.append(get_neighborhood_sum(Z,2,2)) 


        running_state_transitions.extend(txn_vals_after[1:])


print('The running state transition is:                 ',running_state_transitions, '\n The sum of neighbors at that tranisiton is: ', sum)
data = pd.DataFrame({'list1': running_state_transitions, 'list2': sum})
grangercausalitytests(data,1)


BaseLine Cell:  355
15 14
The running state transition is:                  [3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 1, 2, 3, 1, 2, 3, 4, 5, 6, 7, 8, 9, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9, 2, 1, 2, 3, 1, 2, 1, 2, 3, 1, 9, 2, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 3, 4, 5, 6, 7, 8, 1, 9, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 6, 7, 1, 2, 3, 4, 5, 6, 1, 2, 1, 9, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 9, 9, 9, 8, 9, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 2, 3, 4, 5, 1, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 6, 7, 8, 9, 9, 1, 9, 5, 6, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 4, 1, 9, 6, 7, 8, 9, 1, 2, 3, 1, 2, 3, 4, 5, 6, 4, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 9, 7, 8, 9, 3, 4, 1, 2, 3, 1, 9, 7, 8, 9, 7, 8, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6,

{1: ({'ssr_ftest': (0.1245194834193744, 0.7242486349596304, 1142.0, 1),
   'ssr_chi2test': (0.12484659239508203, 0.7238362802229652, 1),
   'lrtest': (0.1248397864819708, 0.7238434996824645, 1),
   'params_ftest': (0.12451948341907086, 0.7242486349599, 1142.0, 1.0)},
   array([[0., 1., 0.]])])}

In [438]:

# #Let's take a simple Python list, create an identical copy, and then use statsmodels to calculate the Granger causality score between the two lists. Here’s how you can do it:


# import pandas as pd
# from statsmodels.tsa.stattools import grangercausalitytests

# # Sample list
# list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10,10,10,11,12,13,14,14,15,15,15,15,15]
# list2 = [1, 2, 3, 5, 5, 6, 7, 8, 9, 10, 10,10,10,11,12,13,14,14,15,15,15,199,13]
# # #list1 = [6, 7, 8, 7, 8, 7, 7, 8, 5, 6, 7, 5, 6, 7, 5, 6, 7, 5, 6, 7, 6, 5, 6, 7, 7, 6, 7, 4, 5, 6, 7, 5, 6, 7]
# # # Identical list
# # list2 = list1.copy()

# # # Combine the data into a DataFrame
# data = pd.DataFrame({'list1': list1, 'list2': list2})

# # Perform Granger causality test
# # maxlag is the maximum number of lags to test for causality. Adjust as needed.
# maxlag = 3  # Choose the number of lags based on your data
# granger_result = grangercausalitytests(data[['list1', 'list2']], 1)

#389 --> 18,18

baseline_cell_to_matrix(199)

(9, 8)